In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

In [2]:
# KL(q || p) = E_{z ~ q}[log q(z) - log p(z)]

KL divergence is always non-negative and 0 if and only if the distributions p and q are equal almost everywhere. Also, note that it is not symmetrical. 

In [4]:
scale_tril = tfb.FillScaleTriL()([-0.5, 1.25, 1.])
scale_tril

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.3132716, 0.       ],
       [1.25     , 0.474087 ]], dtype=float32)>

In [6]:
p = tfd.MultivariateNormalTriL(loc=0, scale_tril=scale_tril)
p

<tfp.distributions.MultivariateNormalTriL 'MultivariateNormalTriL' batch_shape=[] event_shape=[2] dtype=float32>

We are using the `tfb.FillScaleTriL` bijector to build the lower triangular matrix for the multivariate normal tril distribution. It takes, for example, a length 3 vector and produces a 2 by 2 lower triangular matrix with positive entries on the diagonal.

In [7]:
# tfb.Chain([
#        tfb.TransformDiagonal(tfb.Chain([tfb.Shift(1e-5), tfb.SoftPlus()])),
#        tfb.FillTriangular()])
#])

This is what the `tfb.FillScaleTriL` is performing behind the scenes. It first takes the elements of the vector that we pass in and stores them in the entries of a lower triangular matrix using the fill triangular bijector. This bijector fills the elements in a clockwise spiral. Then it makes sure that the diagonal elements are positive. It does this by using the `TransformDiagonal` bijector that applies a transformation to the diagonal entries of the matrix. The bijector that is used is the softplus bijector that maps the real line to the positive reals. Then, it also applies a small shift, to ensure that all the diagonal elements are bounded away from 0, which helps to avoid numerical issues. The `FillScaleTriL` is then a useful way to initialize a covariance matrix. A valid covariance matrix needs to be symmetric and positive definite.

This approach is also useful when you want to learn the covariance matrix of a multivariate Gaussian. If a tensorflow variable is passed through this bijector then the covariance matrix will always be valid, no matter what values the variable takes.

In [9]:
q = tfd.MultivariateNormalDiag(loc=[0.,0.])

This is a simply isotropic Gaussian with zero mean.

In [10]:
tfd.kl_divergence(q, p)

<tf.Tensor: shape=(), dtype=float32, numpy=3.0560925>

This function computes the analytical expression of the KL divergence between q and p assuming that an analytical expression exists. For two Gaussians the expression can be computed analytically.

In [11]:
q = tfd.MultivariateNormalDiag(
    loc=tf.Variable(tf.random.normal([2])),
    scale_diag=tfp.util.TransformedVariable(tf.random.uniform([2]), bijector=tfb.Exp()))

Here let's think of p as the target density that we are interested in approximating. We will be using the KL divergence as an objective function to train our parameterized distribution. The mean and standard deviation of q are set to be learnable tensorflow variables. The mean has been defined to the random initialized according to a zero mean isotropic Gaussian with dimension 2. The scale_diag parameter is also a length 2 variable, but we need to ensure that it remains positive. The `TransformedVariable` class can be used to ensure that. It has two required arguments, the first is the initial value of the tensorflow variable (here initialized from a uniform dist), the second is a bijector object that should be applied to that distribution. Here we are sing the exponential bijector, which means that the scale_diag paramters will be defined by exponentiating the tensorflow variables, so they will always be positive. 

In [12]:
@tf.function
def loss_and_grads(q_dist):
    with tf.GradientTape() as tape:
        loss = tfd.kl_divergence(q_dist, p)
    return loss, tape.gradient(loss, q_dist.trainable_variables)

Our loss and grads function takes the q distribution as an argument and computes the loss as the KL divergence between the q distribution and the target p distribution. For the loss computation we are using the KL divergence function. 

In [ ]:
opt = tf.keras.optimizers.Adam()
for i in range(num_train_steps):
    loss, grads = loss_and_grads(q)
    opt.apply_gradients(zip(grads, q.trainable_variables))

After running our training loop, the paramters of the distribution q will try to match the paramters of the distribution p as close as possible.

Notice that our distribution p is a full covariance Gaussian distribution, whereas the approximate distribution q is a diagonal Gaussian. That means that is impossible for q to match p perfectly, because the family of paramterized distributions from which q comes from does not contain p. This is a very common situation when applying the principle of variational inference, where we choose a paramterized family of distribution to approximate a posterior distribution.

This optimization procedure will give different results based on the order of the distributions when computing the KL divergence. The intuition is that when we are computing KL(q||p) the expectation is taken over q. The KL divergence blows up if there are regions of the event space where q is positive but p is closed to zero. In other words the support of the distribution q is encourage to be in the support of the distribution p. It does not mean that there has to be a perfect match, there are still regions where p is positive and q is close to zero. The KL divergence ignores these regions as it is the expectation over q. If we change the order, the parameters for q will be found for the case where the support of p lies in the support of q. The order KL(q,p) is the order used in Variational Inference. The above explanation gives the fundamentals to why VI often leads to underestimate of the variance of the posterior distribution q.